In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import ray
from ray import tune
from sklearn.model_selection import train_test_split
import yaml
import argparse
import shutil
from collections import namedtuple
import os
import datetime

# Custom Libraries

from utils.data_generator import DataGenerator
from utils.agent import TrainModel
from utils.plotter import Plotter
from utils.helper import to_python_native, gen_experiment_name, set_seed

/Users/qmlwizard/Research/greedy_kernel_alignment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
backend = 'qiskit'
if backend == 'qiskit':
        from utils.qiskit.model import Qkernel
elif backend == 'pennylane':
        from utils.pennylane.model import Qkernel

In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"MPS is available. Using device: {device}")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available. Using device: {device}")
else:
    device = torch.device("cpu")
    print(f"Neither MPS nor CUDA is available. Using CPU: {device}")

MPS is available. Using device: mps


In [4]:
# Read Configs
with open('configs/comparision/checkerboard.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [5]:
data_generator = DataGenerator(     
                                dataset_name = 'Censos Income', 
                                file_path = 'data/adult.npy',
                                n_samples = 500, 
                                noise = 0.1, 
                                num_sectors = 3, 
                                points_per_sector = 10, 
                                grid_size = 4, 
                                sampling_radius = 0.05,
                                n_pca_features=None
                              )

training_data, training_labels, testing_data, testing_labels = data_generator.generate_dataset() #train_test_split(features, target, test_size=0.50, random_state=42)
training_data = torch.tensor(training_data.to_numpy(), dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(testing_data.to_numpy(), dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(training_labels.to_numpy(), dtype=torch.int)
testing_labels = torch.tensor(testing_labels.to_numpy(), dtype=torch.int)

ValueError: could not convert string to float: 'Private'

In [ ]:
kernel = Qkernel(   
                    device = config['qkernel']['device'], 
                    n_qubits = 5, 
                    trainable = True, 
                    input_scaling = True, 
                    data_reuploading = True, 
                    ansatz = 'he', 
                    ansatz_layers = 5   
                )
    
agent = TrainModel(
                    kernel=kernel,
                    training_data=training_data,
                    training_labels=training_labels,
                    testing_data=testing_data,
                    testing_labels=testing_labels,
                    optimizer= 'gd',
                    lr= 0.1,
                    mclr= 0.01,
                    cclr= 0.01,
                    epochs = 50,
                    train_method= 'ccka',
                    target_accuracy=0.95,
                    get_alignment_every=10,  
                    validate_every_epoch=None, 
                    base_path='.',
                    lambda_kao=0.01,
                    lambda_co=0.01,
                    clusters=4
                )

Epochs:  5


In [ ]:
after_metrics = agent.evaluate(testing_data, testing_labels)
after_metrics

/Users/qmlwizard/Research/greedy_kernel_alignment/utils/qiskit/model.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(kernel_value, dtype=torch.float32)


Labels Type:  <class 'torch.Tensor'>


/Users/qmlwizard/Research/greedy_kernel_alignment/utils/qiskit/model.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(kernel_value, dtype=torch.float32)


{'alignment': tensor(0.2699),
 'executions': None,
 'training_accuracy': 1.0,
 'testing_accuracy': 0.992,
 'f1_score': 0.992,
 'alignment_arr': [],
 'loss_arr': [],
 'validation_accuracy_arr': []}

In [ ]:
trained_kernel, main_centroids, class_centroids = agent.fit_kernel(training_data, training_labels)

Started Training


/Users/qmlwizard/Research/greedy_kernel_alignment/utils/qiskit/model.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(kernel_value, dtype=torch.float32)


------------------------------------------------------------------
Epoch: 0th, Alignment: 0
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 1th, Alignment: 0
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 2th, Alignment: 0
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 3th, Alignment: 0
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 4th, Alignment: 0
------------------------------------------------------------------


In [ ]:
after_metrics = agent.evaluate(testing_data, testing_labels)
after_metrics


Labels Type:  <class 'numpy.ndarray'>


/Users/qmlwizard/Research/greedy_kernel_alignment/utils/qiskit/model.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(kernel_value, dtype=torch.float32)


{'alignment': tensor(0.2766),
 'executions': 400,
 'training_accuracy': 1.0,
 'testing_accuracy': 0.996,
 'f1_score': 0.996000064001024,
 'alignment_arr': [0, 0, 0, 0, 0],
 'loss_arr': [],
 'validation_accuracy_arr': []}